<a href="https://colab.research.google.com/github/Duarbert/SAD_DuarteSantos/blob/master/TPC5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



</br></br>

# Sistemas de Apoio à Decisão
#### TPC 5 - PARTE 1 DO TRABALHO FINAL (15 dias)

---------------


##### Com base no seu dataset (Determinado para a sua resolução do Trabalho final)

- Catalogar o dataset quanto a
    - Dimensionalidade
    - Esparsidade
    - Resolução
    - Tamanho


- Catalogar todos as features (Máx 50), quanto a:
    - valores de valores médios, variâncias,histogramas (crie um módulo que faça isto por si);
    - Tipo (contínuo, discreto ou binário)   
    - Analise NaN's (ou Nulls) e Outliers
    - Analise os resultados
    - Quais não fazem sentido em termos do problema que pretende resolver;
    - Que outras features poderia utilizar para enriquecer a informação do seu problema;
    - Perceba se deve ou não fazer encoding ou one hot encoding para cada feature discreta;
    - Faça uma matriz de correlação entre as features usando ```dataframe.corr()```;



- Analise em termos estatísticos a Label/target que tem para já como problema.

---

No final, faça um pequeno texto relatando o dataset que está a utilzar, recorrendo à informação que obteve nas perguntas anteriores;


#Definição

Este dataset tem como objectivo classificar uma dada instância de um empregado de uma empresa como potêncial candidato a sair da empresa por variadíssimas razões.

*Attrition, in Human Resource terminology, refers to the phenomenon of the employees leaving the company. Attrition in a company is usually measured with a metric called attrition rate, which simply measures the no of employees moving out of the company (voluntary resigning or laid off by the company). Attrition Rate is also referred as churn rate or turnover.*

*High attrition is a cause of concern for a company as it presents a cost to the company. The company loses on the amount it spent to recruit and select these employees and to train them for their respective jobs. The company may also have to spend additional money to fill the vacancies left open by these employees. Hence it becomes critical for a company to keep a tab on the attrition rate which down-sizes the employee base.*

**Attrition Rate Formula**

Attrition Rate (%)= (Number of separations/ Number of employees) * 100

*Attrition rate gives an idea as to how many employees are leaving the company at any given time period. It is an important factor as companies have to prepare to start recruiting for the positions which are critical and cannot be left vacant.*

#Dependências#

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
import seaborn

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

# Importar os classificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#Carregar os Dados#

In [0]:
data = pd.DataFrame(pd.read_csv('/content/gdrive/My Drive/SAD/IBMHR.csv', index_col = 9))

#Informação do Dataset - Raw

In [0]:
print('!!! A feature EmployeeNumber está a ser considerada como índice, pois não há nem faz sentido haver duplicados nesta feature !!!\n')
display(data.head())

pd.options.display.max_columns = None
data.columns = data.columns.str.strip()
print('\n')
data.info()

#Catalogar o Dataset#

In [0]:
display('Dimensionalidade = O dataset tem ' + str(len(data.columns)) + ' features / colunas')
print('\n')

display('Tamanho(instâcias, features) = ' + str(data.shape))
print('\n')

display('Resolução do datset na seguinte tabela:')
dataRes = pd.DataFrame(pd.read_csv('/content/gdrive/My Drive/SAD/IBMHR_Features - Catalog.csv'))
display(dataRes)

In [0]:
data.info()

#Pré-processamento dos Dados

In [0]:
#funçao de encode para binarização

def encode_target(df, target_column):
   df_mod = df.copy()
   targets = df_mod[target_column].unique()
   map_to_int = {name: n for n, name in enumerate(targets)}
   df_mod["Target"] = df_mod[target_column].replace(map_to_int)

   return (df_mod, targets)

In [0]:
df2, targets = encode_target(data, "Attrition")
print("* df2.head()", df2[["Target", "Attrition"]].head(),
      sep="\n", end="\n\n")
print("* df2.tail()", df2[["Target", "Attrition"]].tail(),
      sep="\n", end="\n\n")
print("* targets", targets, sep="\n", end="\n\n")

#Análise Estatística de Features#

###Valores Estatísticos

In [0]:
# Easy mode:
# display(data.describe())


# Like a pro
values = [data.mean(), data.var(), data.std()]

result = pd.concat(values,axis=1, join_axes=[media.index])

result.columns = ["Média","Variância","Desvio Padrão"]

result

###Histogramas

In [0]:
# Função para geração dos histogramas.
# Usando a keyword 'All', corre todas as features

def plotHist(datasetVar, columnVar):
    if columnVar == 'All':
      for i in datasetVar.columns:
        if is_numeric_dtype(datasetVar[i]):
          display('You are about to see data from feature ' + i)
          datasetVar.hist(column = i, figsize=[12,4])
          plt.show()
          df3=df2[i].groupby([df2['Target']])
          plt.ylabel('Number of Workers')
          df3.plot(kind='hist', legend = True, title = i, alpha=0.5, figsize=[12,4])
          plt.show()
    else:
        if is_numeric_dtype(datasetVar[columnVar]):
          display('You are about to see data from feature ' + columnVar)
          datasetVar.hist(column = columnVar, figsize=[12,4])
          plt.show()
          df3=df2[columnVar].groupby([df2['Target']])
          plt.ylabel('Number of Workers')
          df3.plot(kind='hist', legend = True, title = columnVar, alpha=0.5, figsize=[12,4])
          plt.show()

plotHist(df2,'All') 


#NaNs e Outliers##


In [164]:
display('Existem ' + str(data.isnull().sum().sum()) + ' valores nulos no dataset')
print('\n')

'Existem 0 valores nulos no dataset'

#Matriz de Correlação

In [0]:
data.corr()

#Teste de Aplicação de Algoritmos

In [192]:
#Decision Tree Classifier

y = df2['Attrition']

features = list(df2.columns[30:32])

x = df2[features]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 5)

parameters = {'criterion':('gini', 'entropy'), 
              'min_samples_split':[2,3,4,5], 
              'max_depth':[9,10,11,12],
              'class_weight':('balanced', None),
              'presort':(False,True),
             }


tr = DecisionTreeClassifier()
gsearch = GridSearchCV(tr, parameters)
gsearch.fit(x_train, y_train)
model = gsearch.best_estimator_
model.score(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8418367346938775

In [0]:
#Random Forest
